### Setup
First, load the model.

In [10]:
from modeling_gpt2 import GPT2Model
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", cache_dir="hf_home/")
model = GPT2Model.from_pretrained("gpt2", cache_dir="hf_home/")
model.cuda()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

### Initial Testing
Now, let's do forward passes on some sample inputs. Starting with the baseline.

In [19]:
inputs = tokenizer(["Today is"], return_tensors="pt").to('cuda')
model.config._attn_implementation = "sdpa"  # NOTE: This is default, but we set manually here for emphasis.
out = model.forward(inputs['input_ids'])
out.last_hidden_state

default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])


tensor([[[ 0.0502,  0.0018, -0.1750,  ..., -0.1020, -0.0257, -0.1292],
         [-0.2410, -0.0911,  0.2592,  ...,  0.4394,  0.3465,  0.1077]]],
       device='cuda:0', grad_fn=<ViewBackward0>)

Now, using our attention implementation. Default is `sdpa`.

In [14]:
model.config._attn_implementation = "minimal_attn"

In [20]:
out = model.forward(inputs['input_ids'])
out.last_hidden_state

default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])
default: torch.Size([1, 2, 12, 64])


tensor([[[ 0.0502,  0.0018, -0.1750,  ..., -0.1020, -0.0257, -0.1292],
         [-0.2410, -0.0911,  0.2592,  ...,  0.4394,  0.3465,  0.1077]]],
       device='cuda:0', grad_fn=<ViewBackward0>)

Great! We can see the shapes are the same and the output tensors are too. This means the attention implementation is correct. Now, let's see if it is faster.

### Generation
Let's try generation now before starting the benchmarking.

In [ ]:
# NOTE: No generation for now bc some way the model works. Probably need to set up with AutoModel but can't figure out for now how to do that and retain our version of modeling_gpt2.py
# outputs = model.generate(
#     **inputs,
#     max_new_tokens=5,
#     return_dict_in_generate=True,
#     output_scores=True,
#     do_sample=False,  # temperature = 0.0 so deterministic
# )

# May need for batching
# tokenizer.pad_token_id = tokenizer.eos_token_id